In [212]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

## Import the Data and Scale it

In [214]:
from numpy import genfromtxt
data = genfromtxt('./data/heston_data.csv', delimiter=',')
X_data = data[:,1:7]
Y_data = data[:,7:8]
print(data.shape)
print(X_data[:10,:])
print(Y_data[100:110,:])


(10000, 8)
[[1.81446697 1.69514594 0.23344394 0.93130717 0.6        1.        ]
 [1.44983173 1.10746649 0.85976295 0.40053882 0.68888889 1.        ]
 [1.7666316  1.71051698 0.70575099 0.13505729 0.77777778 1.        ]
 [1.82521489 1.80922847 0.41492783 0.24089687 0.86666667 1.        ]
 [1.41364921 1.30895837 0.16157663 0.57510016 0.95555556 1.        ]
 [1.26361537 1.48464321 0.33966877 0.22447105 1.04444444 1.        ]
 [1.18141694 1.06354944 0.50344806 0.3184523  1.13333333 1.        ]
 [1.15077381 1.86352486 0.20038426 0.3034765  1.22222222 1.        ]
 [1.73628145 1.79548983 0.20375642 0.87855946 1.31111111 1.        ]
 [1.57764063 1.49032974 0.7886722  0.31360185 1.4        1.        ]]
[[0.39332298]
 [0.30376669]
 [0.2061172 ]
 [0.13443928]
 [0.04994106]
 [0.00097702]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]]


In [215]:
from sklearn.preprocessing import MinMaxScaler

In [216]:
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X_data)
Y_scaled = scaler.fit_transform(Y_data)

### Create Placeholders and Constants

In [225]:
num_inputs = X_data.shape[1]
num_outputs = Y_data.shape[1]
learning_rate = 0.1
num_steps = 20000
batch_size = 10
num_neurons = 50

X = tf.placeholder(tf.float32, [None, num_inputs])
y = tf.placeholder(tf.float32, [None, num_outputs])

In [226]:
#input: X,y,batch_size
#output: X_batch,y_batch: randomly chosen rows from X,y 
def next_batch(X,y,batch_size):
    max_int = X.shape[0]
    batch_ind = np.random.randint(0,max_int,size=batch_size)
    X_batch = X[batch_ind,:]
    y_batch = y[batch_ind,:]
    
    return X_batch,y_batch

In [257]:
from tensorflow.contrib.layers import fully_connected

#Layers
hidden1 = fully_connected(X, num_neurons, activation_fn=tf.nn.elu)
hidden2 = fully_connected(hidden1, num_neurons, activation_fn=tf.nn.elu)
hidden3 = fully_connected(hidden2, num_neurons, activation_fn=tf.nn.elu)
outputs = fully_connected(hidden3, num_outputs, activation_fn=None)

In [258]:
#Loss Function
loss = tf.reduce_mean(tf.square(outputs - y))  # MSE

In [259]:
#Optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

In [260]:
init = tf.global_variables_initializer()

In [261]:
saver = tf.train.Saver()

In [263]:
with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(num_steps):
        
        X_batch,Y_batch = next_batch(X_scaled,Y_scaled,batch_size)
        sess.run(train,feed_dict={X: X_batch, y: Y_batch})
        
        if iteration % 1000 == 0:
            
            mse = loss.eval(feed_dict={X: X_batch, y: Y_batch})
            print(iteration, "\tMSE:", rmse)
    
    saver.save(sess, "./models/heston_nn")

0 	MSE: 24.460344
1000 	MSE: 0.0093858745
2000 	MSE: 0.020631924
3000 	MSE: 0.005541728
4000 	MSE: 0.0066178427
5000 	MSE: 0.03706355
6000 	MSE: 0.020877618
7000 	MSE: 0.01726223
8000 	MSE: 0.07138455
9000 	MSE: 0.027390976
10000 	MSE: 0.024923641
11000 	MSE: 0.039085098
12000 	MSE: 0.033158164
13000 	MSE: 0.04584609
14000 	MSE: 0.025711846
15000 	MSE: 0.040501945
16000 	MSE: 0.041333985
17000 	MSE: 0.059114
18000 	MSE: 0.09262218
19000 	MSE: 0.033234574
